### HYBIRD SEARCH LANGCHAIN

In [1]:
import os
api_key = os.getenv("PINECONE_API_KEY")

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [6]:
from pinecone import Pinecone, ServerlessSpec

index_name = "hybrid-search-langchain-pinecone"
### Initialize Pinecone client
pc = Pinecone(api_key=api_key)

### Create Pinecone index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # Dimension of dense vectors
        metric="dotproduct", ## sparse values 
        spec=ServerlessSpec(cloud='aws', region='us-east-1'),
    )



In [7]:
index = pc.Index(index_name)
index

v:\Genai_course\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
### Vector embedding and sparse matrix
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [9]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

In [13]:
sentences = [
    "In 2023, the global AI market is projected to reach $1.5 trillion.",
    "The advancements in machine learning have revolutionized various industries.",
    "Natural language processing enables computers to understand human language.",
]

##tf-idf values on these sentences
bm25_encoder.fit(sentences)

bm25_encoder.dump("bm25_encoder.json")

bm25_encoder = BM25Encoder().load("bm25_encoder.json")

100%|██████████| 3/3 [00:00<00:00, 26.44it/s]


In [14]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [15]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x00000240D63591F0>, index=<pinecone.db_data.index.Index object at 0x000002409B02B830>)

In [16]:
retriever.add_texts(texts=sentences)

100%|██████████| 1/1 [00:04<00:00,  4.75s/it]


In [18]:
retriever.invoke("What is the market size of AI in 2023?")

[Document(metadata={'score': 0.0612649918}, page_content='Natural language processing enables computers to understand human language.'),
 Document(metadata={'score': 0.588983536}, page_content='In 2023, the global AI market is projected to reach $1.5 trillion.'),
 Document(metadata={'score': 0.188476324}, page_content='The advancements in machine learning have revolutionized various industries.')]